In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import SGDClassifier, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

import pandas as pd
from utility import *

In [ ]:
X, y = load_data()
enc = OneHotEncoder(sparse=False)
y = enc.fit_transform(y)
X_imputed = SimpleImputer().fit_transform(X)

In [ ]:
y_nn = ((data[['ordinalHR']]).to_numpy()*2)
#Split X and Y into training and Test Sets
# y_nn = OneHotEncoder(sparse=True).fit_transform(y_nn)
x_train_nn, x_test_nn, y_train_nn, y_test_nn = train_test_split(X_imputed, y_nn, shuffle=True)

In [ ]:
nn = k.models.Sequential([
    k.layers.Flatten(),
    k.layers.Dense(41, activation='relu'),
    k.layers.Dense(75, activation='relu'),
    k.layers.Dropout(0.3),
    k.layers.Dense(3, activation='softmax'),
])

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-12-14 18:49:05.237860: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-14 18:49:05.238103: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
learning_rate=0.01

nn.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=k.optimizers.Adam(learning_rate),
    metrics=['accuracy']
)
nn(x_train_nn)
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (3135, 94)                0         
                                                                 
 dense (Dense)               (3135, 41)                3895      
                                                                 
 dense_1 (Dense)             (3135, 75)                3150      
                                                                 
 dropout (Dropout)           (3135, 75)                0         
                                                                 
 dense_2 (Dense)             (3135, 3)                 228       
                                                                 
Total params: 7,273
Trainable params: 7,273
Non-trainable params: 0
_________________________________________________________________


In [ ]:
epochs=10
batch_size=50
nn.fit(x_train_nn, y_train_nn, epochs=epochs, batch_size=batch_size)

Epoch 1/10


2022-12-14 18:49:05.520135: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


 1/63 [..............................] - ETA: 23s - loss: 8.7578 - accuracy: 0.2000

2022-12-14 18:49:05.741291: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


63/63 [==============================] - 1s 7ms/step - loss: 1.9449 - accuracy: 0.5107
Epoch 2/10
63/63 [==============================] - 0s 7ms/step - loss: 0.9532 - accuracy: 0.5649
Epoch 3/10
63/63 [==============================] - 0s 6ms/step - loss: 0.9478 - accuracy: 0.5640
Epoch 4/10
63/63 [==============================] - 0s 6ms/step - loss: 0.9473 - accuracy: 0.5630
Epoch 5/10
63/63 [==============================] - 0s 6ms/step - loss: 0.9435 - accuracy: 0.5659
Epoch 6/10
63/63 [==============================] - 0s 6ms/step - loss: 0.9462 - accuracy: 0.5515
Epoch 7/10
63/63 [==============================] - 0s 6ms/step - loss: 0.9375 - accuracy: 0.5633
Epoch 8/10
63/63 [==============================] - 0s 6ms/step - loss: 0.9452 - accuracy: 0.5636
Epoch 9/10
63/63 [==============================] - 0s 6ms/step - loss: 0.9392 - accuracy: 0.5620
Epoch 10/10
63/63 [==============================] - 0s 6ms/step - loss: 0.9388 - accuracy: 0.5707


In [ ]:
print(nn.evaluate(x_test_nn, y_test_nn))

22/33 [===================>..........] - ETA: 0s - loss: 0.9308 - accuracy: 0.5554

2022-12-14 18:49:10.032160: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 5ms/step - loss: 0.9325 - accuracy: 0.5550
[0.9324570298194885, 0.5550239086151123]
